In [1]:
import os
import json
from google.cloud import storage
import pandas as pd

In [2]:
#bucket name
bucket_name = 'flavor-forecast-raw-data'

# initialize a GCS client
storage_client = storage.Client()

# get the right bucket
bucket = storage_client.bucket(bucket_name)

In [3]:
storage_filename = "yelp-dataset/yelp_academic_dataset_business.json"
local_filename = "yelp_business_raw.json"

In [4]:
blob = bucket.blob(storage_filename)
blob.download_to_filename(local_filename)

In [5]:
# creating a json file path to take the input form
json_file_path = "yelp_business_raw.json"

#reading the json file
df_business_raw = pd.read_json(json_file_path, lines=True)

# creating a destination path
csv_file_path = "yelp_business_raw.csv"

#converting the output into a csv file
df_business_raw.to_csv(csv_file_path, index=False)

In [6]:
df_business_raw.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [7]:
# using storage filename for reviews raw dataset
storage_filename_review = "yelp-dataset/yelp_academic_dataset_review.json"
local_filename_review = "yelp_dataset_review_raw.json"

# downloading the dataset
blob = bucket.blob(storage_filename_review)
blob.download_to_filename(local_filename_review)

In [8]:
# creating a json file path to take the input form
json_file_path_review = "yelp_dataset_review_raw.json"

#reading the json file
df_review_raw = pd.read_json(json_file_path_review, lines=True)

# creating a destination path
csv_file_path_review = "yelp_review_raw.csv"

#converting the output into a csv file
df_review_raw.to_csv(csv_file_path_review, index=False)

In [9]:
df_review_raw.shape

(6990280, 9)

In [13]:
reviews = df_review_raw.text

reviews.head()

0    If you decide to eat here, just be aware it is...
1    I've taken a lot of spin classes over the year...
2    Family diner. Had the buffet. Eclectic assortm...
3    Wow!  Yummy, different,  delicious.   Our favo...
4    Cute interior and owner (?) gave us tour of up...
Name: text, dtype: object

In [14]:
reviews_list = list(reviews)
len(reviews_list)

In [21]:
reviews_list_lower = [review.strip().lower() for review in reviews_list]

In [23]:
reviews_list_lower

["if you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. we have tried it multiple times, because i want to like it! i have been to it's other locations in nj and never had a bad experience. \n\nthe food is good, but it takes a very long time to come out. the waitstaff is very young, but usually pleasant. we have just had too many experiences where we spent way too long waiting. we usually opt for another diner or restaurant on the weekends, in order to be done quicker.",
 "i've taken a lot of spin classes over the years, and nothing compares to the classes at body cycle. from the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nfor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nthere is no way i can write this review without giving russell, t

In [25]:
import string

In [28]:
for review in reviews_list_lower:
    for punctuation in string.punctuation:
        review = review.replace(punctuation,'')

In [29]:
reviews_list_lower

["if you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. we have tried it multiple times, because i want to like it! i have been to it's other locations in nj and never had a bad experience. \n\nthe food is good, but it takes a very long time to come out. the waitstaff is very young, but usually pleasant. we have just had too many experiences where we spent way too long waiting. we usually opt for another diner or restaurant on the weekends, in order to be done quicker.",
 "i've taken a lot of spin classes over the years, and nothing compares to the classes at body cycle. from the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nfor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nthere is no way i can write this review without giving russell, t